In [86]:
import folium
import pandas as pd
import geopandas as gpd

In [87]:
import pandas
url = "http://localhost:5001/generate"
data = pandas.read_json(url)

In [88]:
m = folium.Map(location=[48.853,2.35], tiles="OpenStreetMap", zoom_start=12)

In [89]:
for index, row in data.iterrows():
    folium.Marker([row['lat'],row['lon']], popup='<i>'+str(row['id_roi'])+'</i>', tooltip=str(row['id_roi']), icon=folium.Icon()).add_to(m)

In [90]:
m

## Une inégalité de structures en fonction des régions

On regarde combien de structures par région

In [10]:
data['Région'].value_counts()

Auvergne-Rhône-Alpes          21
Île-de-France                 20
Grand Est                     20
Occitanie                     16
Nouvelle-Aquitaine            15
Provence-Alpes-Côte d'Azur    14
Hauts-de-France               14
Normandie                     14
Bourgogne-Franche-Comté       10
Pays de la Loire              10
Bretagne                       9
Centre-Val de Loire            9
Name: Région, dtype: int64

On transforme cela en dataframe

In [11]:
stat_regions = pd.DataFrame(data['Région'].value_counts().reset_index().values, columns=["Region", "Nb"])

In [12]:
stat_regions.head(12)

,Region,Nb
0,Auvergne-Rhône-Alpes,21
1,Île-de-France,20
2,Grand Est,20
3,Occitanie,16
4,Nouvelle-Aquitaine,15
5,Provence-Alpes-Côte d'Azur,14
6,Hauts-de-France,14
7,Normandie,14
8,Bourgogne-Franche-Comté,10
9,Pays de la Loire,10


On charge un fichier geojson représentant le contour des différentes régions de France. Fichier récupéré ici : https://github.com/gregoiredavid/france-geojson

In [13]:
regions = gpd.read_file('regions-version-simplifiee.geojson')

On l'affiche, il contient la description géographique pour chacune des régions

In [14]:
regions

,code,nom,geometry
0,11,Île-de-France,"POLYGON ((2.59052 49.07965, 2.63327 49.10838, ..."
1,24,Centre-Val de Loire,"POLYGON ((2.87463 47.52042, 2.88845 47.50943, ..."
2,27,Bourgogne-Franche-Comté,"POLYGON ((3.62942 46.74946, 3.57569 46.74952, ..."
3,28,Normandie,"POLYGON ((-1.11962 49.35557, -1.07822 49.38849..."
4,32,Hauts-de-France,"POLYGON ((4.04797 49.40564, 4.03991 49.39740, ..."
5,44,Grand Est,"POLYGON ((4.23316 49.95775, 4.30810 49.96952, ..."
6,52,Pays de la Loire,"MULTIPOLYGON (((-2.45849 47.44812, -2.45343 47..."
7,53,Bretagne,"MULTIPOLYGON (((-3.65914 48.65921, -3.63649 48..."
8,75,Nouvelle-Aquitaine,"MULTIPOLYGON (((1.41519 46.34722, 1.43538 46.3..."
9,76,Occitanie,"MULTIPOLYGON (((1.78613 42.57362, 1.77316 42.5..."


On créé une seconde carte pour nos besoins, toujours centré au même endroit

In [15]:
m2 = folium.Map(location=[46.759281,2.401860], tiles="OpenStreetMap", zoom_start=6)

On créé une carte choroplèthe (voir ici : https://fr.wikipedia.org/wiki/Carte_choropl%C3%A8the) 
<br>On fait un join entre les données géographiques et le dataframe contenant le nombre de structures par régions (avec un identifiant unique qui est le nom de la région). On fait un dégradé de couleur pour afficher les différences entre régions

In [16]:
m2.choropleth(
    geo_data=regions,
    name="Statistiques",
    data=stat_regions,
    columns=['Region','Nb'],
    key_on='feature.properties.nom',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Statistiques Régions"
    )

/usr/local/lib/python3.7/dist-packages/folium/folium.py:415: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  FutureWarning
/usr/local/lib/python3.7/dist-packages/pyproj/crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))


On affiche la seconde carte

In [17]:
m2